In [1]:
from glob import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import shutil
from torchvision import transforms
import torch
from torchvision.datasets import ImageFolder
from sklearn.model_selection import StratifiedKFold
import cv2
%matplotlib inline

In [2]:
def extract_patches_2d(img,patch_shape,step=[1.0,1.0],batch_first=False):
    patch_H, patch_W = patch_shape[0], patch_shape[1]
    if(img.size(2)<patch_H):
        num_padded_H_Top = (patch_H - img.size(2))//2
        num_padded_H_Bottom = patch_H - img.size(2) - num_padded_H_Top
        padding_H = nn.ConstantPad2d((0,0,num_padded_H_Top,num_padded_H_Bottom),0)
        img = padding_H(img)
    if(img.size(3)<patch_W):
        num_padded_W_Left = (patch_W - img.size(3))//2
        num_padded_W_Right = patch_W - img.size(3) - num_padded_W_Left
        padding_W = nn.ConstantPad2d((num_padded_W_Left,num_padded_W_Right,0,0),0)
        img = padding_W(img)
    step_int = [0,0]
    step_int[0] = int(patch_H*step[0]) if(isinstance(step[0], float)) else step[0]
    step_int[1] = int(patch_W*step[1]) if(isinstance(step[1], float)) else step[1]
    patches_fold_H = img.unfold(2, patch_H, step_int[0])
    if((img.size(2) - patch_H) % step_int[0] != 0):
        patches_fold_H = torch.cat((patches_fold_H,img[:,:,-patch_H:,].permute(0,1,3,2).unsqueeze(2)),dim=2)
    patches_fold_HW = patches_fold_H.unfold(3, patch_W, step_int[1])   
    if((img.size(3) - patch_W) % step_int[1] != 0):
        patches_fold_HW = torch.cat((patches_fold_HW,patches_fold_H[:,:,:,-patch_W:,:].permute(0,1,2,4,3).unsqueeze(3)),dim=3)
    patches = patches_fold_HW.permute(2,3,0,1,4,5)
    patches = patches.reshape(-1,img.size(0),img.size(1),patch_H,patch_W)
    if(batch_first):
        patches = patches.permute(1,0,2,3,4)
    return patches

In [30]:
batch_size = 1

transform = transforms.Compose([transforms.Resize((478,478))
                                       ,transforms.ToTensor()])

data = ImageFolder('C:/Users/yonsei/Desktop/asd',transform=transform)
trainloader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=False, num_workers=5, pin_memory=True)
index = 0
for batch_idx, (inputs, labels) in enumerate(trainloader):
    img = extract_patches_2d(inputs,[299,299],step=[0.6,0.6],batch_first=True)
    for i in range(16):
        index = index +1
        for j in range(4):
            img1 = img[i][j]
            img1 = img1.numpy()
            img1 = img1.transpose(1, 2, 0)
            image = (img1 * 255).round().astype(np.uint8)
            image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
            #cv2.imwrite('D:/capstone/split_patient/S1/data/5fold/train/benign/benign_%d_%d.jpg'%(index,j),image )
            cv2.imwrite('C:/Users/yonsei/Desktop/asd/benign/259-2_%d.jpg'%(j+1),image)
       

IndexError: index 1 is out of bounds for dimension 0 with size 1